In [1]:
# # In this activity, more challenging than the last, 
#     you will build a Twitter bot that sends out 
#     visualized sentiment analysis of a Twitter 
#     account's recent tweets.

# # Visit https://twitter.com/PlotBot5 for an example 
#     of what your script should do.

# # The bot receives tweets via mentions and in turn 
#     performs sentiment analysis on the most recent 
#     twitter account specified in the mention

# # For example, when a user tweets, "@PlotBot Analyze: 
#     @CNN," it will trigger a sentiment analysis on 
#     the CNN twitter feed.

# # A plot from the sentiment analysis is then tweeted 
#     to the PlotBot5 twitter feed. See below for 
#     examples of scatter plots you will generate:





# # Hints, requirements, and considerations:


# # Your bot should scan your account every five minutes 
#     for mentions.
# # Your bot should pull 500 most recent tweets to 
#     analyze for each incoming request.
# # Your script should prevent abuse by analyzing only 
#     Twitter accounts that have not previously been 
#     analyzed.
# # Your plot should include meaningful legend and labels.
# # It should also mention the Twitter account name of 
#     the requesting user.
# # When submitting your assignment, be sure to have at 
#     least three analyses tweeted out from your account 
#     (enlist the help of classmates, friends, or family, 
#      if necessary!).
# # Notable libraries used to complete this application 
#     include: Matplotlib, Pandas, Tweepy, and VADER.
# # You may find it helpful to organize your code in 
#     function(s), then call them.
# # If you're not yet familiar with creating functions 
#     in Python, here is a tutorial you may wish to 
#     consult: https://www.tutorialspoint.com/python/python_functions.htm.

In [2]:
# Dependencies
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Target Term
target_term = "@CWPlotBot"

# Counter
counter = 1

# Variables for holding sentiments
sentiments = []

target_user_list = []

# Target User Account
target_user = []

# # Variables for holding sentiments
#     compound_list = []
#     positive_list = []
#     negative_list = []
#     neutral_list = []

# Variable for max_id
oldest_tweet = None

In [4]:
# def ScanForMentions():
#     # Search for all tweets
#     public_tweets = api.search(target_term, count=100, result_type="recent")

#     # Loop through all tweets
#     for tweet in public_tweets["statuses"]:

#         # Get ID and Author of most recent tweet directed to me
# # don't need this??         tweet_id = tweet["id"]
#         tweet_author = tweet["user"]["screen_name"]
#         target_user.append(tweet["entities"]["user_mentions"][1]["screen_name"])
        
#         if target_user not in target_user_list:
#             target_user_list.append(tweet_mention)
#             TweetSentiment()
#             time.sleep(300)

In [5]:
def TweetSentiment():
    
    # Loop through 25 pages of tweets (total 500 tweets)
#     for x in range(1, 26):
    for x in range(25):
        # Get all tweets from home feed
        public_tweets = api.user_timeline(target_user, page=x)

        # Loop through all tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
        
            # Get Tweet ID, subtract 1, and assign to oldest_tweet
            oldest_tweet = tweet['id'] - 1
        
            # Add sentiments for each tweet into a list
            sentiments.append({"Date": tweet["created_at"], 
                               "Compound": compound,
                               "Positive": pos,
                               "Negative": neu,
                               "Neutral": neg,
                               "Tweets Ago": counter})
            
            # Add to counter 
            counter += 1
        
            # Convert sentiments to DataFrame
            sentiments_pd = pd.DataFrame.from_dict(sentiments)
            sentiments_pd.head()

# plot results 
            # Create plot
            x_vals = sentiments_pd["Tweets Ago"]
            y_vals = sentiments_pd["Compound"]
            plt.plot(x_vals,
                     y_vals, marker="o", linewidth=0.5,
                     alpha=0.8)

            # # Incorporate the other graph properties
            now = datetime.now()
            now = now.strftime("%d/%m/%Y")
#             now = now.strftime("%Y-%m-%d %H:%M")
            plt.title(f"Sentiment Analysis of Tweets ({now}) for {target_user}")
            plt.xlim([x_vals.max(),x_vals.min()])
            plt.ylabel("Tweet Polarity")
            plt.xlabel("Tweets Ago")
#             plt.show()

            # Save the figure
            plt.savefig("SentimentAnalysis.png")
            
# tweet results
        api.update_with_media("SentimentAnalysis.png", "New Tweet Analysis: @" + tweet_mention + " (Thx @" + tweet_author + "!!)")

        # Use Try-Except to avoid the duplicate error
#         try:
#             # Tweet results
            
# #             file = open('Your_image.png', 'rb')
# #             data = file.read()
# # #             api.request('statuses/update_with_media', {'status':f"New Tweet Analysis: {tweet_mention} (Thnx @{tweet_author}!)"}, 
# #                         {'media[]':data})
#             api.update_with_media("SentimentAnalysis.png",
#                     "New Tweet Analysis: @" + tweet_mention + " (Thx @" + tweet_author + "!!)")
# #             api.update_status(
# #                 f"New Tweet Analysis: {tweet_mention} (Thnx @{tweet_author}!)")
# #           ADD PLOT TO TWEET

#             # Print success message
#             print("Successful response!")

#         except Exception:            # Print message if duplicate
#             print("Already responded to this one!")

#         # Print message to signify complete cycle
#         print("We're done for now. I'll check again in 5 minutes.")

In [6]:
# def ScanForMentions():
#     # Search for all tweets
#     public_tweets = api.search(target_term, count=100, result_type="recent")

#     # Loop through all tweets
#     for tweet in public_tweets["statuses"]:

#         # Get ID and Author of most recent tweet directed to me
# # don't need this??         tweet_id = tweet["id"]
#         tweet_author = tweet["user"]["screen_name"]
#         tweet_mention = tweet["entities"]["user_mentions"][1]["screen_name"]
        
#         if tweet_mention not in target_user_list:
#             target_user_list.append(tweet_mention)
#             TweetSentiment()
#             time.sleep(300)

def ScanForMentions():
    # Search for all tweets
    public_tweets = api.search(target_term, count=100, result_type="recent")

    # Loop through all tweets
    for tweet in public_tweets["statuses"]:

        # Get ID and Author of most recent tweet directed to me
# don't need this??         tweet_id = tweet["id"]
        tweet_author = tweet["user"]["screen_name"]
        target_user.append(tweet["entities"]["user_mentions"][1]["screen_name"])
        
        if target_user not in target_user_list:
            target_user_list.append(target_user)
            TweetSentiment()
            time.sleep(300)

In [7]:
ScanForMentions()

TweepError: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]